## Este código é responsável por ler a resposta da API do site do STF e converter para um catálogo em csv tratado.

#### Query de busca:

 "consequencialismo" ou "tragédia dos comuns" ou "natureza da firma" ou "eficiência de Pareto" ou "produtos rivais" ou "produtos não-rivais" ou "barreiras de entrada" ou "custo de oportunidade" ou "teorema de Coase" ou "bens públicos" ou "perda social" ou "meios alternativos de solução de conflitos" ou "análise econômica da litigância" ou "teoria da escolha racional" ou "teoria dos jogos" ou "custos sociais" ou "economia comportamental" ou "seleção adversa" ou "externalidades negativas" ou "captura regulatória" ou "enforcement" ou "falhas de mercado" ou "excedente do consumidor" ou "assimetria regulatória" ou "sistema de incentivos" ou "custos de transação" ou "trade-off" ou "teoria da escolha pública" ou "assimetria de informações" ou "problema do agente-principal" ou "eficiência de Kaldor–Hicks" ou "monopólio legal" ou "Posner" ou "Coase" ou "Cooter" ou "Ulen" ou "Shavell" ou "North" ou "Mackaay" ou "Análise econômica do direito"

In [1]:
import json
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')


pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 100)

In [2]:
def process_list_field(field_value):
    """Processa campos que podem ser listas, convertendo para string"""
    if isinstance(field_value, list):
        return '; '.join([str(item) for item in field_value if item is not None])
    return field_value

def clean_text_field(text):
    """Limpa campos de texto removendo quebras de linha excessivas"""
    if isinstance(text, str):
        return text.replace('\n', ' ').replace('\r', ' ').strip()
    return text

def truncate_text(text, max_length=1000):
    """Trunca texto se for muito longo"""
    if text and isinstance(text, str) and len(text) > max_length:
        return text[:max_length] + '...'
    return text

In [ ]:
# Caminho do arquivo JSON
json_file = "merged_result.json"

try:
    with open(json_file, 'r', encoding='utf-8') as file:
        data = json.load(file)
    print(f" Estrutura do JSON:")
    print(f"Chaves principais: {list(data.keys())}")
    
    if 'result' in data:
        result_keys = list(data['result'].keys())
        print(f"   - Chaves em 'result': {result_keys}")
        
        if 'hits' in data['result']:
            total_hits = data['result']['hits']['total']['value']
            actual_hits = len(data['result']['hits']['hits'])
            print(f"   - Total de documentos: {total_hits}")
            print(f"   - Documentos retornados: {actual_hits}")

except FileNotFoundError:
    print(f" Erro: Arquivo '{json_file}' não encontrado.")
except json.JSONDecodeError as e:
    print(f"Erro ao decodificar JSON: {e}")

 Estrutura do JSON:
Chaves principais: ['result', 'search_id']
   - Chaves em 'result': ['took', 'timed_out', '_shards', 'hits', 'aggregations']
   - Total de documentos: 1109
   - Documentos retornados: 1109


In [ ]:
documents = data['result']['hits']['hits']
processed_docs = []

for i, doc in enumerate(documents):
    source = doc['_source']
    
    processed_doc = {
        # Identificação
        'id': source.get('id'),
        'titulo': source.get('titulo'),
        'base': source.get('base'),
        'score': doc.get('_score'),
        
        # Processo
        'processo_codigo_completo': source.get('processo_codigo_completo'),
        'processo_numero': source.get('processo_numero'),
        'processo_classe': source.get('processo_classe_processual_unificada_extenso'),
        'processo_classe_sigla': source.get('processo_classe_processual_unificada_classe_sigla'),
        
        # Localização
        'procedencia_uf': source.get('procedencia_geografica_uf_sigla'),
        'procedencia_uf_extenso': source.get('procedencia_geografica_uf_extenso'),
        'procedencia_completo': source.get('procedencia_geografica_completo'),
        
        # Ministros
        'ministro_facet': process_list_field(source.get('ministro_facet')),
        'relator_processo': source.get('relator_processo_nome'),
        'relator_acordao': source.get('relator_acordao_nome'),
        
        # Datas
        'julgamento_data': source.get('julgamento_data'),
        'publicacao_data': source.get('publicacao_data'),
        'republicacao_data': source.get('republicacao_data'),
        
        # Órgão julgador
        'orgao_julgador': source.get('orgao_julgador'),
        
        # Tipos de processo
        'is_questao_ordem': source.get('is_questao_ordem'),
        'is_repercussao_geral': source.get('is_repercussao_geral'),
        'is_repercussao_geral_merito': source.get('is_repercussao_geral_merito'),
        'is_colac': source.get('is_colac'),
        'is_processo_antigo': source.get('is_processo_antigo'),
        
        # URLs
        'inteiro_teor_url': source.get('inteiro_teor_url'),
        'acompanhamento_processual_url': source.get('acompanhamento_processual_url'),
        'dje_url': source.get('dje_url'),
        
        # Textos principais
        'ementa_texto': truncate_text(clean_text_field(source.get('ementa_texto')), 1000),
        'acordao_ata': truncate_text(clean_text_field(source.get('acordao_ata')), 500),
        
        # Partes
        'partes_lista_texto': clean_text_field(source.get('partes_lista_texto')),
        
        # Documentação
        'documental_publicacao': clean_text_field(source.get('documental_publicacao_lista_texto')),
        'documental_indexacao': truncate_text(clean_text_field(source.get('documental_indexacao_texto')), 500),
        
        # Tese (se houver)
        'documental_tese_tipo': source.get('documental_tese_tipo'),
        'documental_tese_texto': clean_text_field(source.get('documental_tese_texto')),
        'documental_tese_tema': clean_text_field(source.get('documental_tese_tema_texto')),
        
        # Metadados
        'dg_atualizado_em': source.get('dg_atualizado_em'),
        'ods_onu': process_list_field(source.get('ods_onu'))
    }
    
    processed_docs.append(processed_doc)

print(f"{len(processed_docs)} documentos processados")

1109 documentos processados


In [ ]:
df = pd.DataFrame(processed_docs)
date_columns = ['julgamento_data', 'publicacao_data', 'republicacao_data', 'dg_atualizado_em']
for col in date_columns:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')

print(f" DataFrame criado com {len(df)} linhas e {len(df.columns)} colunas")
print(f" Colunas disponíveis:")
for i, col in enumerate(df.columns, 1):
    print(f"   {i:2d}. {col}")

 DataFrame criado com 1109 linhas e 36 colunas
 Colunas disponíveis:
    1. id
    2. titulo
    3. base
    4. score
    5. processo_codigo_completo
    6. processo_numero
    7. processo_classe
    8. processo_classe_sigla
    9. procedencia_uf
   10. procedencia_uf_extenso
   11. procedencia_completo
   12. ministro_facet
   13. relator_processo
   14. relator_acordao
   15. julgamento_data
   16. publicacao_data
   17. republicacao_data
   18. orgao_julgador
   19. is_questao_ordem
   20. is_repercussao_geral
   21. is_repercussao_geral_merito
   22. is_colac
   23. is_processo_antigo
   24. inteiro_teor_url
   25. acompanhamento_processual_url
   26. dje_url
   27. ementa_texto
   28. acordao_ata
   29. partes_lista_texto
   30. documental_publicacao
   31. documental_indexacao
   32. documental_tese_tipo
   33. documental_tese_texto
   34. documental_tese_tema
   35. dg_atualizado_em
   36. ods_onu


In [9]:
print(" PRIMEIRAS 5 LINHAS DO DATASET")
main_cols = ['titulo', 'processo_classe_sigla', 'procedencia_uf', 'orgao_julgador', 
             'julgamento_data', 'is_repercussao_geral', 'score']
available_cols = [col for col in main_cols if col in df.columns]

display(df[available_cols].head())

print(" INFORMAÇÕES DETALHADAS")
df.info()

 PRIMEIRAS 5 LINHAS DO DATASET


,titulo,processo_classe_sigla,procedencia_uf,orgao_julgador,julgamento_data,is_repercussao_geral,score
0,ADPF 449,ADPF,DF,Tribunal Pleno,2019-05-08,False,193.25310
1,ARE 1267734 AgR-ED-EDv-AgR-AgR,ARE,PR,Tribunal Pleno,2023-11-13,False,187.49574
2,ARE 1324429 AgR-ED,ARE,SC,Tribunal Pleno,2023-11-13,False,180.39906
3,ADI 3995,ADI,DF,Tribunal Pleno,2018-12-13,False,139.86073
4,ADI 7063 ED,ADI,RJ,Tribunal Pleno,2022-11-10,False,123.63258


 INFORMAÇÕES DETALHADAS
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1109 entries, 0 to 1108
Data columns (total 36 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   id                             1109 non-null   object        
 1   titulo                         1109 non-null   object        
 2   base                           1109 non-null   object        
 3   score                          1109 non-null   float64       
 4   processo_codigo_completo       1109 non-null   object        
 5   processo_numero                1109 non-null   int64         
 6   processo_classe                1109 non-null   object        
 7   processo_classe_sigla          1109 non-null   object        
 8   procedencia_uf                 1108 non-null   object        
 9   procedencia_uf_extenso         1108 non-null   object        
 10  procedencia_completo           1109 non-null   object       

In [ ]:
csv_output = "jurisprudencia_data.csv"
df.to_csv(csv_output, index=False, encoding='utf-8-sig')